<a href="https://colab.research.google.com/github/Rookiehhh/AI-NoteBooks/blob/main/llm/pruning_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [8]:
!nvidia-smi

Mon Dec 16 00:50:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0              28W /  70W |    105MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# 查看GPU
print(torch.cuda.get_device_name(0))

Tesla T4


剪枝教程
================

**作者**: [Michela Paganini](https://github.com/mickypaganini)

最先进的深度学习技术依赖于过于参数化的模型，这些模型难以部署。相反，生物神经网络以高效的稀疏连接而闻名。找到通过减少模型中参数数量来压缩模型的最佳技术非常重要，以减少内存、电池，并且……稀疏化你的神经网络，以及如何扩展它以实现你自己的自定义剪枝技术。

要求
------------

`"torch>=1.4.0a0+8e8a5e0"`


In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

创建模型
==============

在本教程中，我们使用了
[LeNet](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf) 架构
来自 LeCun 等人, 1998 年。


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 检查是否可以使用CUDA，如果不能则使用CPU

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1个输入图像通道，6个输出通道，5x5的卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 图像尺寸为5x5
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 最后一层分类，输出10类

    def forward(self, x):
        # 应用第一个卷积层，之后接ReLU激活函数和2x2的最大池化层
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 应用第二个卷积层，之后接ReLU激活函数和2x2的最大池化层
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        # 展平特征图为一维
        x = x.view(-1, self.num_flat_features(x))
        # 应用第一个全连接层
        x = F.relu(self.fc1(x))
        # 应用第二个全连接层
        x = F.relu(self.fc2(x))
        # 应用第三个全连接层，输出分类结果
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 计算张量x中非batch维数的元素总数，用于展平操作
        size = x.size()[1:]  # 除去batch维度的其他维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# 实例化模型并将其移动到设备上（GPU或CPU）
model = LeNet().to(device=device)

检查一个模块
================

让我们检查一下我们 LeNet 模型中的 (未剪枝的) `conv1` 层。它将包含两个参数 `weight` 和 `bias`，目前没有缓冲区。


In [11]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 5.3230e-02, -1.9184e-01, -1.5453e-01, -1.2699e-01,  6.6093e-02],
          [ 1.8400e-01, -9.0329e-02, -1.2064e-01, -1.8656e-01, -1.7463e-01],
          [ 7.2806e-02, -1.5050e-01, -7.3161e-02, -1.1542e-01, -1.0708e-01],
          [ 2.7203e-02, -1.6833e-01, -6.0692e-02, -4.8347e-02, -3.6435e-02],
          [-1.1822e-01,  7.2270e-02,  1.9358e-01, -1.2367e-01, -1.2044e-01]]],


        [[[-1.1181e-01, -3.6317e-03, -3.7799e-02, -2.1182e-02, -1.9294e-01],
          [-9.4081e-02, -1.6163e-01, -1.7939e-01, -1.9383e-02,  1.7369e-01],
          [ 1.8170e-01,  6.0109e-02, -2.9164e-02,  1.2290e-01,  4.4267e-02],
          [-6.1668e-02, -1.8145e-01, -1.8272e-01, -4.7278e-02, -2.5070e-02],
          [-1.5373e-01,  8.0089e-02,  2.5887e-02,  6.4049e-02,  7.2076e-02]]],


        [[[ 3.7609e-02, -1.3780e-01, -5.4621e-02, -1.2206e-02,  5.2841e-02],
          [-5.2555e-02, -1.8912e-01,  6.6217e-02,  2.7229e-02,  2.3559e-03],
          [-5.6309e-02, -1.8930e-0

In [12]:
print(list(module.named_buffers()))

[]


修剪模块
================

要修剪一个模块（在这个例子中，是我们 LeNet 架构的 `conv1` 层），首先从 `torch.nn.utils.prune` 中选择一种可用的修剪技术（或通过继承 `BasePruningMethod` [实现](#extending-torch-nn-utils-pruning-with-custom-pruning-functions) 自己的修剪方法）。然后，指定要修剪的模块及其内部参数的名称。最后，使用所选修剪技术所需的适当关键字参数，指定修剪参数。

在这个例子中，我们将随机修剪 `conv1` 层中名为 `weight` 的参数中的 30% 连接。模块作为函数的第一个参数传递；`name` 通过其字符串标识符识别该模块中的参数；而 `amount` 表示要修剪的连接的百分比（如果是介于 0 和 1 之间的浮点数），或者要修剪的绝对连接数（如果是一个非负整数）。

In [13]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

修剪通过从参数中移除 `weight` 并用一个新的参数 `weight_orig` 替换它（即在初始参数 `name` 后附加 `"_orig"`）。 `weight_orig` 存储张量的未修剪版本。 `bias` 没有被修剪，因此将保持不变。

In [14]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-0.0082, -0.1869, -0.0693,  0.1968, -0.1167,  0.0080], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 5.3230e-02, -1.9184e-01, -1.5453e-01, -1.2699e-01,  6.6093e-02],
          [ 1.8400e-01, -9.0329e-02, -1.2064e-01, -1.8656e-01, -1.7463e-01],
          [ 7.2806e-02, -1.5050e-01, -7.3161e-02, -1.1542e-01, -1.0708e-01],
          [ 2.7203e-02, -1.6833e-01, -6.0692e-02, -4.8347e-02, -3.6435e-02],
          [-1.1822e-01,  7.2270e-02,  1.9358e-01, -1.2367e-01, -1.2044e-01]]],


        [[[-1.1181e-01, -3.6317e-03, -3.7799e-02, -2.1182e-02, -1.9294e-01],
          [-9.4081e-02, -1.6163e-01, -1.7939e-01, -1.9383e-02,  1.7369e-01],
          [ 1.8170e-01,  6.0109e-02, -2.9164e-02,  1.2290e-01,  4.4267e-02],
          [-6.1668e-02, -1.8145e-01, -1.8272e-01, -4.7278e-02, -2.5070e-02],
          [-1.5373e-01,  8.0089e-02,  2.5887e-02,  6.4049e-02,  7.2076e-02]]],


        [[[ 3.7609e-02, -1.3780e-01, -5.462

由上述选定的剪枝技术生成的剪枝掩码作为名为 `weight_mask` 的模块缓冲区保存（即在初始参数 `name` 后附加 `"_mask"`）。


In [15]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1., 1., 0.],
          [1., 0., 1., 1., 0.],
          [0., 1., 0., 1., 1.],
          [1., 0., 0., 1., 1.],
          [1., 0., 0., 0., 1.]]],


        [[[1., 0., 1., 1., 1.],
          [1., 1., 0., 1., 1.],
          [0., 0., 1., 1., 0.],
          [0., 0., 1., 1., 1.],
          [0., 1., 1., 1., 1.]]],


        [[[0., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.],
          [1., 0., 1., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[1., 1., 1., 0., 1.],
          [0., 1., 1., 0., 1.],
          [0., 1., 0., 1., 0.],
          [0., 1., 1., 0., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 1., 0., 1.],
          [1., 1., 0., 1., 1.],
          [1., 0., 1., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 0., 0., 1., 0.]]]], 

为了使前向传播正常工作而不需要修改，必须存在 `weight` 属性。 `torch.nn.utils.prune` 中实现的剪枝技术计算权重的剪枝版本（通过将掩码与原始参数结合）并将其存储在 `weight` 属性中。请注意，这不再是 `module` 的一个参数，它现在仅仅是一个属性。


In [16]:
print(module.weight)

tensor([[[[ 5.3230e-02, -0.0000e+00, -1.5453e-01, -1.2699e-01,  0.0000e+00],
          [ 1.8400e-01, -0.0000e+00, -1.2064e-01, -1.8656e-01, -0.0000e+00],
          [ 0.0000e+00, -1.5050e-01, -0.0000e+00, -1.1542e-01, -1.0708e-01],
          [ 2.7203e-02, -0.0000e+00, -0.0000e+00, -4.8347e-02, -3.6435e-02],
          [-1.1822e-01,  0.0000e+00,  0.0000e+00, -0.0000e+00, -1.2044e-01]]],


        [[[-1.1181e-01, -0.0000e+00, -3.7799e-02, -2.1182e-02, -1.9294e-01],
          [-9.4081e-02, -1.6163e-01, -0.0000e+00, -1.9383e-02,  1.7369e-01],
          [ 0.0000e+00,  0.0000e+00, -2.9164e-02,  1.2290e-01,  0.0000e+00],
          [-0.0000e+00, -0.0000e+00, -1.8272e-01, -4.7278e-02, -2.5070e-02],
          [-0.0000e+00,  8.0089e-02,  2.5887e-02,  6.4049e-02,  7.2076e-02]]],


        [[[ 0.0000e+00, -1.3780e-01, -5.4621e-02, -1.2206e-02,  0.0000e+00],
          [-5.2555e-02, -1.8912e-01,  6.6217e-02,  2.7229e-02,  2.3559e-03],
          [-5.6309e-02, -0.0000e+00,  9.6571e-02, -0.0000e+00, -0.00

最后，在每次前向传递之前应用剪枝，使用 PyTorch 的 `forward_pre_hooks`。具体来说，当 `module` 被剪枝时，正如我们所做的，它将为与之关联的每个被剪枝的参数获取一个 `forward_pre_hook`。在这种情况下，由于到目前为止我们仅剪枝了名为 `weight` 的原始参数，因此只会存在一个钩子。


In [17]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x7f1011dcca30>)])


为了完整起见，我们现在也可以修剪 `bias`，以查看 `module` 的参数、缓冲区、钩子和属性是如何变化的。为了尝试另一种修剪技术，这里我们通过 L1 范数修剪了 `bias` 中的 3 个最小条目，具体实现见 `l1_unstructured` 修剪函数。


In [18]:
prune.l1_unstructured(module, name="bias", amount=3)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

我们现在期望命名参数包括 `weight_orig`（之前的）和 `bias_orig`。缓冲区将包括 `weight_mask` 和 `bias_mask`。这两个张量的剪枝版本将作为模块属性存在，并且模块现在将有两个 `forward_pre_hooks`。


In [19]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 5.3230e-02, -1.9184e-01, -1.5453e-01, -1.2699e-01,  6.6093e-02],
          [ 1.8400e-01, -9.0329e-02, -1.2064e-01, -1.8656e-01, -1.7463e-01],
          [ 7.2806e-02, -1.5050e-01, -7.3161e-02, -1.1542e-01, -1.0708e-01],
          [ 2.7203e-02, -1.6833e-01, -6.0692e-02, -4.8347e-02, -3.6435e-02],
          [-1.1822e-01,  7.2270e-02,  1.9358e-01, -1.2367e-01, -1.2044e-01]]],


        [[[-1.1181e-01, -3.6317e-03, -3.7799e-02, -2.1182e-02, -1.9294e-01],
          [-9.4081e-02, -1.6163e-01, -1.7939e-01, -1.9383e-02,  1.7369e-01],
          [ 1.8170e-01,  6.0109e-02, -2.9164e-02,  1.2290e-01,  4.4267e-02],
          [-6.1668e-02, -1.8145e-01, -1.8272e-01, -4.7278e-02, -2.5070e-02],
          [-1.5373e-01,  8.0089e-02,  2.5887e-02,  6.4049e-02,  7.2076e-02]]],


        [[[ 3.7609e-02, -1.3780e-01, -5.4621e-02, -1.2206e-02,  5.2841e-02],
          [-5.2555e-02, -1.8912e-01,  6.6217e-02,  2.7229e-02,  2.3559e-03],
          [-5.6309e-02, -1.89

In [20]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1., 1., 0.],
          [1., 0., 1., 1., 0.],
          [0., 1., 0., 1., 1.],
          [1., 0., 0., 1., 1.],
          [1., 0., 0., 0., 1.]]],


        [[[1., 0., 1., 1., 1.],
          [1., 1., 0., 1., 1.],
          [0., 0., 1., 1., 0.],
          [0., 0., 1., 1., 1.],
          [0., 1., 1., 1., 1.]]],


        [[[0., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.],
          [1., 0., 1., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[1., 1., 1., 0., 1.],
          [0., 1., 1., 0., 1.],
          [0., 1., 0., 1., 0.],
          [0., 1., 1., 0., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 1., 0., 1.],
          [1., 1., 0., 1., 1.],
          [1., 0., 1., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 0., 0., 1., 0.]]]], 

In [21]:
print(module.bias)

tensor([-0.0000, -0.1869, -0.0000,  0.1968, -0.1167,  0.0000], device='cuda:0',
       grad_fn=<MulBackward0>)


In [22]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x7f1011dcca30>), (1, <torch.nn.utils.prune.L1Unstructured object at 0x7f1011dcf1c0>)])


迭代剪枝  
=================

同一个模块中的参数可以被多次剪枝，各次剪枝调用的效果等同于依次应用的多个掩码的组合。新掩码与旧掩码的组合由 `PruningContainer` 的 `compute_mask` 方法处理。

例如，假设我们现在希望进一步剪枝 `module.weight`，这次使用基于张量第0轴的结构化剪枝（第0轴对应卷积层的输出通道，对于 `conv1`，其维度为6），剪枝依据通道的L2范数。这可以通过 `ln_structured` 函数实现，其中 `n=2` 且 `dim=0`。



In [23]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# 正如我们所验证的，这将使对应于50%（6个通道中的3个）的所有连接归零, 同时保留先前掩码的作用。
print(module.weight)

tensor([[[[ 0.0532, -0.0000, -0.1545, -0.1270,  0.0000],
          [ 0.1840, -0.0000, -0.1206, -0.1866, -0.0000],
          [ 0.0000, -0.1505, -0.0000, -0.1154, -0.1071],
          [ 0.0272, -0.0000, -0.0000, -0.0483, -0.0364],
          [-0.1182,  0.0000,  0.0000, -0.0000, -0.1204]]],


        [[[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000,

对应的钩子现在将是类型 `torch.nn.utils.prune.PruningContainer`，并将存储应用于 `weight` 参数的剪枝历史。


In [24]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # 选择正确的钩子
        break

print(list(hook))  # 在容器中的剪枝历史

[<torch.nn.utils.prune.RandomUnstructured object at 0x7f1011dcca30>, <torch.nn.utils.prune.LnStructured object at 0x7f1011dcf610>]


序列化修剪后的模型
==========================

所有相关的张量，包括掩码缓冲区和用于计算修剪张量的原始参数都存储在模型的`state_dict`中，因此可以很容易地序列化并保存（如果需要的话）。

In [25]:
print(model.state_dict().keys())

odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


移除剪枝重参数化
=================

为了使剪枝永久生效，移除关于 `weight_orig` 和 `weight_mask` 的重参数化，并移除 `forward_pre_hook`，我们可以使用 `torch.nn.utils.prune` 中的 `remove` 功能。请注意，这并不会撤销剪枝，好像它从未发生过。相反，它通过将参数 `weight` 重新分配给模型参数的剪枝版本，使其永久化。

在去除重新参数化之前：

In [26]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 5.3230e-02, -1.9184e-01, -1.5453e-01, -1.2699e-01,  6.6093e-02],
          [ 1.8400e-01, -9.0329e-02, -1.2064e-01, -1.8656e-01, -1.7463e-01],
          [ 7.2806e-02, -1.5050e-01, -7.3161e-02, -1.1542e-01, -1.0708e-01],
          [ 2.7203e-02, -1.6833e-01, -6.0692e-02, -4.8347e-02, -3.6435e-02],
          [-1.1822e-01,  7.2270e-02,  1.9358e-01, -1.2367e-01, -1.2044e-01]]],


        [[[-1.1181e-01, -3.6317e-03, -3.7799e-02, -2.1182e-02, -1.9294e-01],
          [-9.4081e-02, -1.6163e-01, -1.7939e-01, -1.9383e-02,  1.7369e-01],
          [ 1.8170e-01,  6.0109e-02, -2.9164e-02,  1.2290e-01,  4.4267e-02],
          [-6.1668e-02, -1.8145e-01, -1.8272e-01, -4.7278e-02, -2.5070e-02],
          [-1.5373e-01,  8.0089e-02,  2.5887e-02,  6.4049e-02,  7.2076e-02]]],


        [[[ 3.7609e-02, -1.3780e-01, -5.4621e-02, -1.2206e-02,  5.2841e-02],
          [-5.2555e-02, -1.8912e-01,  6.6217e-02,  2.7229e-02,  2.3559e-03],
          [-5.6309e-02, -1.89

In [27]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1., 1., 0.],
          [1., 0., 1., 1., 0.],
          [0., 1., 0., 1., 1.],
          [1., 0., 0., 1., 1.],
          [1., 0., 0., 0., 1.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 0.]]],


        [[[0., 1., 1., 0., 1.],
          [1., 1., 0., 1., 1.],
          [1., 0., 1., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 0., 0., 1., 0.]]]], 

In [28]:
print(module.weight)

tensor([[[[ 0.0532, -0.0000, -0.1545, -0.1270,  0.0000],
          [ 0.1840, -0.0000, -0.1206, -0.1866, -0.0000],
          [ 0.0000, -0.1505, -0.0000, -0.1154, -0.1071],
          [ 0.0272, -0.0000, -0.0000, -0.0483, -0.0364],
          [-0.1182,  0.0000,  0.0000, -0.0000, -0.1204]]],


        [[[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000,

去除重新参数化后：


In [29]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias_orig', Parameter containing:
tensor([-0.0082, -0.1869, -0.0693,  0.1968, -0.1167,  0.0080], device='cuda:0',
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0532, -0.0000, -0.1545, -0.1270,  0.0000],
          [ 0.1840, -0.0000, -0.1206, -0.1866, -0.0000],
          [ 0.0000, -0.1505, -0.0000, -0.1154, -0.1071],
          [ 0.0272, -0.0000, -0.0000, -0.0483, -0.0364],
          [-0.1182,  0.0000,  0.0000, -0.0000, -0.1204]]],


        [[[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0

In [30]:
print(list(module.named_buffers()))

[('bias_mask', tensor([0., 1., 0., 1., 1., 0.], device='cuda:0'))]


在模型中修剪多个参数
=====================

通过指定所需的修剪技术和参数，我们可以轻松地修剪网络中的多个张量，可能根据它们的类型，正如我们在这个例子中将看到的。

In [31]:
new_model = LeNet()
for name, module in new_model.named_modules():
    # 在所有 2D 卷积层中修剪 20% 的连接
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # 在所有线性层中修剪 40% 的连接
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # 验证所有掩码是否存在

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])


全局剪枝
==============

到目前为止，我们只看到了通常所称的“局部”剪枝，即逐个修剪模型中的张量，通过将每个条目的统计数据（权重大小、激活、梯度等）与该张量中的其他条目进行比较。然而，一种常见且可能更强大的技术是一次性修剪整个模型，例如，移除整个模型中最低的20%的连接，而不是在每一层中去除最低的20%的连接。这可能导致每一层的剪枝比例不同。让我们看看如何使用 `torch.nn.utils.prune` 中的 `global_unstructured` 来实现这一点。

In [33]:
model = LeNet()

# 定义需要进行剪枝的参数
parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

# 全局非结构化剪枝，使用L1范数作为剪枝方法，剪枝比例为0.2
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # 使用L1非结构化剪枝方法
    amount=0.2,  # 剪枝20%的权重
)

现在我们可以检查每个剪枝参数所诱导的稀疏性，这在每一层中不会等于20%。但是，整体稀疏性将是（大约）20%。

In [34]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 8.00%
Sparsity in conv2.weight: 13.88%
Sparsity in fc1.weight: 22.16%
Sparsity in fc2.weight: 12.07%
Sparsity in fc3.weight: 11.55%
Global sparsity: 20.00%


扩展 `torch.nn.utils.prune` 自定义修剪函数
==============================================================

要实现您自己的修剪函数，可以通过继承 `BasePruningMethod` 基类来扩展 `nn.utils.prune` 模块，方法与所有其他修剪方法相同。基类为您实现了以下方法：`__call__`，`apply_mask`，`apply`，`prune` 和 `remove`。除了某些特例外，您无需重新实现这些方法。不过，您需要实现 `__init__`（修剪参数的…）。

假设您想实现一种修剪技术，该技术修剪张量中的每个其他条目（或者如果张量之前已被修剪，则修剪张量中剩余的未修剪部分）。这将属于 `PRUNING_TYPE='unstructured'`，因为它作用于层中的单个连接，而不是整个单元/通道（`'structured'`），或在不同的参数之间（`'global'`）。

In [35]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """剪枝张量中每隔一个位置的元素"""
    PRUNING_TYPE = 'unstructured'  # 非结构化剪枝

    def compute_mask(self, t, default_mask):
        # 计算掩码
        mask = default_mask.clone()  # 克隆默认掩码
        mask.view(-1)[::2] = 0  # 将掩码每隔一个元素置为0，实现隔位剪枝
        return mask

现在，在将其应用于`nn.Module`中的参数时，你还需要提供一个简单的函数，用于实例化该方法并应用它。


In [36]:
def foobar_unstructured(module, name):
    """通过移除张量中每隔一个位置的元素，对`module`中名为`name`的参数执行剪枝操作。
    此操作会直接修改模块本身（同时返回修改后的模块），具体操作包括：
    1) 添加一个名为`name+'_mask'`的缓冲区，对应剪枝方法应用于参数`name`的二进制掩码。
    参数`name`会被其剪枝后的版本替代，而原始（未剪枝）的参数会存储在一个新参数
    `name+'_orig'`中。

    示例:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)  # 应用FooBarPruningMethod剪枝方法
    return module  # 返回修改后的模块

让我们试试看！


In [37]:
model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
